# 3D Gaussian Splatting

Before starting, go to `Runtime -> Change Runtime Type` and set Hardware Acceslerator to `T4 GPU` (or something similar).

The developed Gaussian Splatting project ([here](https://github.com/graphdeco-inria/gaussian-splatting)) is not compatible with Colab default settings. So, first we have to setup the environment:
1. Python 3.7
2. CUDA 11.8 toolkit
3. PyTorch 1.12.1+cu116

## Install Python 3.7

In [ ]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py37_23.1.0-1-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y python=3.7
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages')
!python --version  # Should say Python 3.7.x

## Install CUDA 11.8

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
!chmod +x cuda_11.8.0_520.61.05_linux.run
!./cuda_11.8.0_520.61.05_linux.run --silent --toolkit --no-drm --no-man-page
import os
os.environ['PATH'] += ':/usr/local/cuda-11.8/bin'
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-11.8/lib64:/usr/lib64-nvidia'
!nvcc --version  # Should show CUDA 11.8

## Install Pytorch

In [ ]:
!pip uninstall torch torchvision torchaudio -y
!pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu116

In [ ]:
import torch
print(torch.cuda.is_available())  # Should be True
print(torch.version.cuda)        # Should be 11.3 (from PyTorch)
print(torch.cuda.get_device_name(0))  # Should show GPU
!nvidia-smi

## Clone the Guassian Splatting [repository](https://github.com/camenduru/gaussian-splatting)

In [ ]:
%cd /content
!git clone --recursive https://github.com/camenduru/gaussian-splatting
!pip install -q plyfile

%cd /content/gaussian-splatting
!pip install -q /content/gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -q /content/gaussian-splatting/submodules/simple-knn

## Copy COLMAP Results to GS environment
Replace the path with your image path (```/content/drive/MyDrive/COLMAP Results```)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r "/content/drive/MyDrive/COLMAP Results" "/content"

!mkdir -p /content/project/undistorted/sparse/0
!mv /content/project/undistorted/sparse/*.bin /content/project/undistorted/sparse/0/

Mounted at /content/drive


## Train Gaussian Splats

In [ ]:
!python train.py -s /content/project/undistorted/

### Render GS

Change `ec6fd63d-9` with the output folder name you see in the results of `train.py` script.

In [ ]:
!python render.py \
  -s /content/project/undistorted \
  -m /content/gaussian-splatting/output/ec6fd63d-9

After performing the rendering, download the `*.ply` file from the output folder, go to this [website](https://playcanvas.com/) and view the final results.